In [ ]:
import datetime
import urllib
from os.path import dirname, join
import os

import pandas as pd

from bokeh.io import curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, DataRange1d, Select, RadioButtonGroup, TextAreaInput, Paragraph
from bokeh.palettes import Blues4
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

output_notebook()

Define a function that will tell the Bokeh which port to serve our plot on. This is only necessary when running in a cloud-hosted JupyterHub environment.

In [ ]:
def remote_jupyter_proxy_url(port):
    """
    Callable to configure Bokeh's show method when a proxy must be
    configured.

    If port is None we're asking about the URL
    for the origin header.
    """
    #base_url = os.environ['EXTERNAL_URL']
    base_url = 'https://jupyterhub.cuahsi.org'
    host = urllib.parse.urlparse(base_url).netloc
    
    # If port is None we're asking for the URL origin
    # so return the public hostname.
    if port is None:
        return host

    service_url_path = os.environ['JUPYTERHUB_SERVICE_PREFIX']
    proxy_url_path = 'proxy/%d' % port

    user_url = urllib.parse.urljoin(base_url, service_url_path)
    full_url = urllib.parse.urljoin(user_url, proxy_url_path)
    
    print(full_url)
    return full_url

Create our Bokeh plot. All code must be within a parent function (e.g. `bkapp`) for the plot to be rendered properly. This means that all event handler functions are nested within the parent function.

In [ ]:


def bkapp(doc):

    # enable/disable debugging output
    debug = True
    


    def log(value):
        if debug:
            text = debug_box.value
            text += value + '\n'
            debug_box.value = text
    
    def update_plot(attrname, old, new):
        global df
        
#         #df = pd.read_csv('dat/covid-deaths.tsv',
        df = pd.read_csv('covid-deaths.tsv',
                         delimiter='\t')
    
        
        state = state_select.value
        
        
        log(f'update_plot: {state}')
        
        # set plot title
        select_dataset_label = data_select.labels[data_select.active]       
        plot.title.text = f'{select_dataset_label}: {state}'
        
        
#        plot.title.text = f"{plot_title}{state}"
        source.data = dict(x=dates, y=df[state])
    
        update_style(None, None, style_select.value)
        update_mode(None, None, mode_select.value)
        update_agg(None, None, agg_select.value)

    def update_style(attrname, old, new):
        style = new.lower()
        if style == 'line':
            line.visible = True
            vbar.visible = False
        if style == 'bar':
            line.visible = False
            vbar.visible = True

    def update_mode(attrname, old, new):
        global df
        
        agg = new.lower()
        state = state_select.value
        if agg == 'cumulative':
            # plot raw cumulative data
            source.data = dict(x=dates, y=df[state])
        if agg == 'incremental':
            # compute incremental
            diffs = df[state].diff()
            diffs[diffs < 0] = 0
            source.data = dict(x=dates,
                               y=diffs)
        if agg == 'percent change':
            # compute percent change
            pct = df[state].diff().pct_change()
            pct = pct.fillna(0)
            source.data = dict(x=dates,
                               y=pct)

    def update_agg(attrname, old, new):
        # create new dataframe from current x,y data
        # then perform date agg
        df = pd.DataFrame(source.data)
        df = df.set_index('x')

        agg = agg_select.value
        state = state_select.value

        
        if agg == 'weekly':
            df = df.groupby(pd.TimeGrouper('W')).sum().dropna()
        if agg == 'daily':
            df = df.groupby(pd.TimeGrouper('D')).sum().dropna()

        source.data = dict(x=df.index, y=df.y)


    def update_data(attrname, old, new):
        global df, plot_title
        selected = data_select.labels[new]
        if selected == 'Confirmed':
            df = pd.read_csv('dat/covid-confirmed.tsv',
                             delimiter='\t')
            plot_title = 'COVID Confirmed: ' 
        else:
            df = pd.read_csv('dat/covid-deaths.tsv',
                             delimiter='\t')
            plot_title = 'COVID Deceased: ' 
        update_plot('', '', '')
    
    
    # read our data
    df = pd.read_csv('covid-deaths.tsv',
                         delimiter='\t')
    
    plot = figure(x_axis_type="datetime",
                  plot_width=800, toolbar_location="below")
    source = ColumnDataSource()

           
    mode = 'cumulative'
    style= 'line'
    agg = 'daily'
    state = 'Massachusetts'



    states = df.columns[1:]
    state_select = Select(value=state,
                          title='State',
                          options=sorted(states))
    mode_select = Select(value='Cumulative',
                         title='Mode',
                         options=['Cumulative',
                                  'Incremental',
                                  'Percent Change'])
    agg_select = Select(value='Daily',
                        title='Aggregation',
                        options=['Daily',
                                 'Weekly',
                                 '7-Day Ave'])
    style_select = Select(value='Line',
                            title='Style',
                            options=['Line',
                                     'Bar'])

    # setting initial data
    dates = pd.to_datetime(df.date, format='%Y%m%d')
    source = ColumnDataSource(data=dict(x=dates,
                                        y=df[state]))

    # create the line plot
    line = plot.line('x', 'y',
                     source=source,
                     line_width=3,
                     line_alpha=0.6)
    
    # create the bar plot and make it hidden
    vbar = plot.vbar(x='x', top='y', width=0.2,
                     source=source)
    vbar.visible = False

    # fixed attributes
    plot_title = 'COVID Deceased: ' 
    plot.xaxis.axis_label = None
    plot.yaxis.axis_label = "Count"
    plot.axis.axis_label_text_font_style = "bold"
    plot.x_range = DataRange1d(range_padding=0.0)
    plot.grid.grid_line_alpha = 0.3

    data_select = RadioButtonGroup(labels=['Confirmed', 'Deceased'], active=1)

    state_select.on_change('value', update_plot)
    style_select.on_change('value', update_style)
    mode_select.on_change('value', update_mode)
    agg_select.on_change('value', update_agg)
    data_select.on_change('active', update_data)

    controls = column(data_select,
                      state_select,
                      mode_select,
                      style_select,
                      agg_select)


    doc.add_root(row(plot, controls))
    
    # add debugging panel
    if debug:
        debug_box = TextAreaInput(value="This is a textbox to capture log messages. Turn off by setting the debug flag to false at the top of application (i.e. debug=False)\n",
                                  rows=10,
                                  title="Debugging Panel",
                                  min_width=800,
                                  disabled=True)
        doc.add_root(row(debug_box))

#     text_output = Paragraph(text='', width=200, height=100)
#     doc.add_root(row(text_output))

    state = state_select.value
    plot.title.text = f"{plot_title}{state}"




Download the latest COVID-19 data

In [ ]:
!python collectdata.py

## Display the Plot

The command below will render our Bokeh app within this Jupyter notebook, and errors messages will not be displayed in the notebook. The best way to debug Bokeh errors on a JupyterHub server is by opening the browser's debugging tools. Instructions for several common browsers are shown below:

#### Google Chrome
- Select in Menu Bar: `View` -> `Developer` -> `Developer Tools`

#### Mozilla Firefox
- Select in Menu Bar: `Tools` -> `Web Developer` -> `Toggle Tools`

#### Apple Safari
- Select in Menu Bar: `Safari` -> `Preferences`
    - Click `Advanced`
    - Select `Show Develop Menu in menu bar`
- Select in Menu Bar: `Develop` -> `Show Web Inspector`

#### Microsoft Edge
- Select in Edge Menu Bar: `...` -> `Developer Tools`
    

In [ ]:
handle = show(bkapp, notebook_url=remote_jupyter_proxy_url)  